# Dashboard for GVOs (Geomagnetic Virtual Observatories)

Initial prototype before integration into VirES server

Product documentation (check links at the right):  
https://www.space.dtu.dk/english/research/projects/project-descriptions/geomagnetic-virtual-observatories

For this notebook, put the file `SW_OPER_VOBS_1M_2_20131215T000000_20200315T000000_0102.cdf` into the running directory, obtained from:  
http://www.spacecenter.dk/files/magnetic-models/GVO/GVO_data_SWARM.zip

In [ ]:
import numpy as np
import cdflib
import xarray as xr
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

import panel as pn
import hvplot.xarray
import holoviews as hv
import geoviews as gv
# import geoviews.feature as gf
hv.extension('bokeh', 'matplotlib')
# gv.extension('bokeh', 'matplotlib')
pn.extension()

## Load in the GVO dataset

In [ ]:
def cdftime_to_datetime(t):
    CDF_EPOCH_1970 = 62167219200000.0
    return np.array(pd.to_datetime((t - CDF_EPOCH_1970), unit='ms'))

def cdf_to_xarray(cdf_file):
    cdf = cdflib.CDF(cdf_file)
    # The data are arranged in blocks of 300 GVO samples (lat/lon positions)
    # We can therefore slice out the time sampling points like:
    t = cdftime_to_datetime(cdf.varget("Timestamp")[0:-1:300])
    t_SV = cdftime_to_datetime(cdf.varget("Timestamp_SV")[0:-1:300])
    # and the lat/lon points are repeated every 300 records, so we can just extract the first block:
    lat = cdf.varget("Latitude")[0:300]
    lon = cdf.varget("Longitude")[0:300]
    # Identify the number of time samples
    len_t = len(t)
    # Construct the xarray.Dataset, reshaping the arrays, and attaching suitable dimension names
    ds = xr.Dataset(
        data_vars={
            "B_OB": (("Timestamp", "gridpt", "rtp"), np.reshape(cdf.varget("B_OB"), (len_t, 300, 3))),
            "sigma_OB": (("Timestamp", "gridpt", "rtp"), np.reshape(cdf.varget("sigma_OB"), (len_t, 300, 3))),
            "B_CF": (("Timestamp", "gridpt", "rtp"), np.reshape(cdf.varget("B_CF"), (len_t, 300, 3))),
            "sigma_CF": (("Timestamp", "gridpt", "rtp"), np.reshape(cdf.varget("sigma_CF"), (len_t, 300, 3))),
            "B_SV": (("Timestamp", "gridpt", "rtp"), np.reshape(cdf.varget("B_SV"), (len_t, 300, 3))),
            "sigma_SV": (("Timestamp", "gridpt", "rtp"), np.reshape(cdf.varget("sigma_SV"), (len_t, 300, 3))),
        },
        coords={
            "Timestamp": t,
            "Timestamp_SV": t_SV,
            "Latitude": ("gridpt", lat),
            "Longitude": ("gridpt", lon),
            "rtp": np.array(["Radial", "Theta", "Phi"])
        },
    )
    # Append metadata
    for var in (list(ds.data_vars) + ["Timestamp", "Timestamp_SV", "Latitude", "Longitude"]):
        ds[var].attrs = cdf.varattsget(var)
    ds["rtp"].attrs = {"DESCRIPTION": "Components - Radial (r), Theta (t), Phi (p)", "UNITS": "-"}
    return ds

# ds = cdf_to_xarray('SW_OPER_VOBS_1M_2_20131215T000000_20200315T000000_0102.cdf')
ds = cdf_to_xarray('SW_OPER_VOBS_4M_2_20140301T000000_20200301T000000_0102.cdf')
# ds = cdf_to_xarray('CH_OPER_VOBS_4M_2_20001101T000000_20100701T000000_0102.cdf')
ds

In [ ]:
# # Now we can easily slice out data from specific GVOs:
# ds.sel(rtp="Radial", gridpt=0)["B_OB"].plot.line(x="Timestamp")

## Plot with cartopy/matplotlib

In [ ]:
GVO_LOCATIONS = np.vstack((ds["Longitude"].values, ds["Latitude"].values)).T

LINE_COLORS = {
    "Radial": mpl.colors.to_hex("tab:blue"),
    "Theta": mpl.colors.to_hex("tab:orange"),
    "Phi": mpl.colors.to_hex("tab:green")
}

In [ ]:
def make_map(rtp="Radial", highlight_gridpt=0):
    color = LINE_COLORS.get(rtp)
    fig = plt.figure(figsize=(8,4))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree(),extent=[-180, 180, -90, 90])
    ax.coastlines()
    ax.scatter(*GVO_LOCATIONS.T, color="black", s=3, zorder=2)
#     mini = ds.sel(rtp="Radial")["B_OB"].data.min()
#     maxi = ds.sel(rtp="Radial")["B_OB"].data.max()
    # Loop through each GVO
    for i in range(300):
        gvo_record = ds["B_OB"].sel(rtp=rtp, gridpt=i)
        # Extract a fractional (0-1) representation of the GVO data
#         gvo_y_scaled = (gvo_record.data - mini) / (maxi - mini)
        # SCALED SEPARATELY FOR EACH GVO
        gvo_y_scaled = (gvo_record.data - np.nanmin(gvo_record.data)) / (np.nanmax(gvo_record.data) - np.nanmin(gvo_record.data))
        # Construct vertices to represent the GVO data, scaled between -5 and 5
        # And centred around the GVO location in lat/lon space
        lat, lon = float(gvo_record.Latitude), float(gvo_record.Longitude)
        xdata = lon + np.linspace(-5, 5, len(gvo_y_scaled))
        ydata = lat + 10*(gvo_y_scaled - 0.5)
        gvo_xy_verts = np.vstack((xdata, ydata))
        # Plot these lines onto the figure
        ax.plot(*gvo_xy_verts, transform=ccrs.PlateCarree(), color=color, alpha=0.8, linewidth=0.8)
    # Draw a circle around selected grid point
    gridpt_latlon = ds[["Latitude", "Longitude"]].sel(gridpt=highlight_gridpt)
    lat, lon = float(gridpt_latlon["Latitude"]), float(gridpt_latlon["Longitude"])
    ax.scatter(
        lon, lat, transform=ccrs.PlateCarree(),
        s=350, facecolors='grey', edgecolors='grey', linewidth=5
    )
    fig.text(0.5, 1, f"Selected LAT={round(lat, 1)}, LON={round(lon, 1)}", fontsize=15, ha="center", va="top")
    return fig

# make_map(rtp="Radial", highlight_gridpt=101);

## Redo map plot with GeoViews

In [ ]:
def wee_lines_for_map(ds, rtp="Radial", paths_or_points="points", param="OB"):
    """Returns GeoViews Paths for all the little lines to go on the map"""
    parameter = f"B_{param}"
    wee_paths = []
    lons = []
    lats = []
    for gridpt in range(300):
        gvo_record = ds[parameter].sel(rtp=rtp, gridpt=gridpt)
        if all(np.isnan(gvo_record.data)):
            continue
        # Extract a fractional (0-1) representation of the GVO data
        #         gvo_y_scaled = (gvo_record.data - mini) / (maxi - mini)
        # SCALED SEPARATELY FOR EACH GVO
        # subsamble the data to make it a bit lighter
        mini, maxi = np.nanmin(gvo_record.data), np.nanmax(gvo_record.data)
#         data = gvo_record.data[::4]
        data = gvo_record.data
        gvo_y_scaled = (data - mini) / (maxi - mini)
        # Screen out nans from this (they break the plotting)
        gvo_y_scaled = gvo_y_scaled[~np.isnan(gvo_y_scaled)]
        # Construct vertices to represent the GVO data, scaled between -5 and 5
        # And centred around the GVO location in lat/lon space
        lat, lon = float(gvo_record.Latitude), float(gvo_record.Longitude)
        xdata = lon + np.linspace(-5, 5, len(gvo_y_scaled))
        ydata = lat + 10*(gvo_y_scaled - 0.5)
        lons.append(xdata)
        lats.append(ydata)
    if paths_or_points=="points":
        # Merge all points from all GVOs to plot as one overlay
        lons = np.concatenate(lons)
        lats = np.concatenate(lats)
        lonlats = np.vstack((lons, lats)).T
        return gv.Overlay([gv.Points(lonlats, ["Longitude", "Latitude"]).opts(color=LINE_COLORS[rtp], size=0.5, alpha=1)], group="GVO_data")
    elif paths_or_points=="paths":
        # Plot data as paths, from each GVO individually (Slow)
        gvo_xy_verts_all = [dict(Longitude=xdata, Latitude=ydata) for xdata, ydata in zip(lons, lats)]
        wee_paths = [
            gv.Path(gvo_xy_verts).opts(color=LINE_COLORS[rtp])
            for gvo_xy_verts in gvo_xy_verts_all
        ]
        return gv.Overlay(wee_paths, group="GVO_data")


class GVOMap():
    
    def __init__(self, rtp="Radial", draw_lines=True, use_pointer=True, param="OB"):
        self.basemap = gv.Overlay(self._basemap_with_GVO_points())
        if use_pointer:
            self.pointer_tap = hv.streams.Tap(source=self.basemap, x=0, y=90)
            self.basemap *= self._highlight_gvo()
        self.update_figure_with_lines(rtp=rtp, draw_lines=draw_lines, param=param)
        # Set the pointer_tap as a property of the hv object
        # so we can easily extract current position with p.object.pointer_tap
        if use_pointer:
            self.object.pointer_tap = self.pointer_tap
    
    def update_figure_with_lines(self, rtp="Radial", draw_lines=True, param="OB"):
        """Provides an updatable figure accessible at self.object"""
        if draw_lines:
            self.object = self.basemap * wee_lines_for_map(ds, rtp=rtp, param=param)
        else:
            self.object = self.basemap
        self.object.opts(
            projection=ccrs.PlateCarree(), aspect=2, global_extent=True, frame_width=500,
#             default_tools=[], tools=[], active_tools=[]  # why doesn't this work?
        )
    
    def _basemap_with_GVO_points(self):
        """Generates list of static GeoViews elements for the underlying map"""
        elements = [
            gv.feature.ocean.opts(alpha=0.7),
            gv.feature.land.opts(alpha=0.7),
        ]
        # Dots for the centres of all GVOs
        elements.append(
            gv.Points(GVO_LOCATIONS).opts(color="black", size=1)
#             .opts(default_tools=[], tools=[], active_tools=[])
        )
        return elements
    
    def _highlight_gvo(self):
        return gv.DynamicMap(
            lambda x, y: gv.Points([self.nearest_GVO(x, y)[:2]]),
            streams=[self.pointer_tap],
        ).opts(projection=ccrs.PlateCarree(), color="grey", size=18, alpha=0.8)

    @staticmethod
    def nearest_GVO(x, y):
        """Given (x, y) location, return (x, y, gridpt) of the nearest GVO"""
        gridpt = int(np.argmin(np.sum((GVO_LOCATIONS - np.array([x, y]))**2, axis=1)))
        x, y = GVO_LOCATIONS[gridpt]
        return x, y, gridpt
    
    def selected_GVO_gridpt(self):
        x, y = self.pointer_tap.x, self.pointer_tap.y
        _, _, gridpt = self.nearest_GVO(x, y)
        return gridpt

# p1 = GVOMap(draw_lines=True, use_pointer=True, rtp="Radial")
# p1.update_figure_with_lines("Phi", param="OB")
# p1.basemap + p1.object

In [ ]:
# print(
#     p1.object.pointer_tap, "\n"
#     "Nearest GVO: ",
#     GVOMap.nearest_GVO(p1.object.pointer_tap.x, p1.object.pointer_tap.y),
# )

In [ ]:
# html = pn.pane.HTML("Clicked: ...")

# # # # watch for changes with .param.watch
# def f(eventx, eventy):
#     x, y = eventx.obj.x, eventx.obj.y
#     x, y, gridpt = GVOMap.nearest_GVO(x, y)
#     html.object = f"Clicked: x:{x}, y:{y}, gridpt:{gridpt}"
# p1.object.pointer_tap.param.watch(f, ["x", "y"])

# html

## HoloViews plot for individual GVO

In [ ]:
def make_gvo_lineplot(gridpt=0, param="OB"):
    """Generate holoviews object showing the three components"""
    
    def _select_obs_and_rename_vars(ds, gridpt, param):
        # Must split apart and rename components so that they are treated as
        # separate entities with shared_axes in hvplot
        # https://discourse.holoviz.org/t/how-to-only-link-share-only-the-x-axis/93
        _ds = ds[[f"B_{param}", f"sigma_{param}"]]
        _ds = _ds.sel(gridpt=gridpt)
        _ds[f"B_{param} (Radial)"] = _ds[f"B_{param}"].sel(rtp="Radial")
        _ds[f"B_{param} (Theta)"] = _ds[f"B_{param}"].sel(rtp="Theta")
        _ds[f"B_{param} (Phi)"] = _ds[f"B_{param}"].sel(rtp="Phi")
        _ds[f"sigma_{param} (Radial)"] = _ds[f"sigma_{param}"].sel(rtp="Radial")
        _ds[f"sigma_{param} (Theta)"] = _ds[f"sigma_{param}"].sel(rtp="Theta")
        _ds[f"sigma_{param} (Phi)"] = _ds[f"sigma_{param}"].sel(rtp="Phi")
#         _ds = _ds.drop_vars(["B_OB", "sigma_OB"])
        _ds = _ds.drop_dims("rtp")
        return _ds

    _ds = _select_obs_and_rename_vars(ds, gridpt, param)
    color_r = LINE_COLORS["Radial"]
    color_t = LINE_COLORS["Theta"]
    color_p = LINE_COLORS["Phi"]
    lineplots = \
          hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_{param} (Radial)", line_color=color_r, xlabel="", grid=True, height=180, width=500) \
            * hvplot.plot(_ds, kind="errorbars", x="Timestamp", y=f"B_{param} (Radial)", yerr1=f"sigma_{param} (Radial)") \
        + hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_{param} (Theta)", line_color=color_t, xlabel="", grid=True, height=180, width=500) \
            * hvplot.plot(_ds, kind="errorbars", x="Timestamp", y=f"B_{param} (Theta)", yerr1=f"sigma_{param} (Theta)") \
        + hvplot.plot(_ds, kind="line", x="Timestamp", y=f"B_{param} (Phi)", line_color=color_p, xlabel="", grid=True, height=180, width=500) \
            * hvplot.plot(_ds, kind="errorbars", x="Timestamp", y=f"B_{param} (Phi)", yerr1=f"sigma_{param} (Phi)")
    lat = float(_ds.Latitude)
    lon = float(_ds.Longitude)
    title = f"Selected LAT={round(lat, 1)}, LON={round(lon, 1)}"
    lineplots.cols(1).opts(shared_axes=True, sizing_mode="stretch_both", title=title)
    return lineplots

# p = make_gvo_lineplot(gridpt=30, param="OB")
# p

## Build dashboard with Panel

In [ ]:
panes = {
    "hv-map": pn.pane.HoloViews(),
    "hv-gvo-line": pn.pane.HoloViews()
}

widgets = {
    "select_cpt": pn.widgets.Select(name="Preview component on map:", options=["Radial", "Theta", "Phi"], width=200),
    "select_param": pn.widgets.Select(name="Select parameter:", options=["OB", "CF", "SV"], width=200)
}

loading_indicator = pn.pane.HTML("")
# pn.indicators.LoadingSpinner(value=False, width=50, height=50)
def set_loading(loading):
    if loading:
        loading_indicator.object = '<p style="color:red; font-size:200%"><strong>Loading...</strong></p>'
    else:
        loading_indicator.object = ''

# Map with tap to select GVO
hv_map = GVOMap(rtp="Radial", draw_lines=True, use_pointer=True, param="OB")
panes["hv-map"].object = hv_map.object
def update_map_rtp(event):
    """Change the selected component shown on the map"""
    set_loading(True)
    hv_map.update_figure_with_lines(rtp=event.obj.value, param=widgets["select_param"].value)
    panes["hv-map"].object = hv_map.object
    set_loading(False)
def update_map_param(event):
    """Change the selected parameter shown on the map"""
    set_loading(True)
    param = event.obj.value
    hv_map.update_figure_with_lines(rtp=widgets["select_cpt"].value, param=param)
    panes["hv-map"].object = hv_map.object
    # Also update the GVO line plots
    gridpt = hv_map.selected_GVO_gridpt()
    panes["hv-gvo-line"].object = make_gvo_lineplot(gridpt, param=param)
    set_loading(False)

# Detailed line plots from GVOs
panes["hv-gvo-line"].object = make_gvo_lineplot(0)
def update_gvo_lines(event):
    """Update the line figure according to the pointer tap on the map"""
    set_loading(True)
    x, y = event.obj.x, event.obj.y
    x, y, gridpt = GVOMap.nearest_GVO(x, y)
    panes["hv-gvo-line"].object = make_gvo_lineplot(gridpt, param=widgets["select_param"].value)
    set_loading(False)

hv_map.object.pointer_tap.param.watch(update_gvo_lines, "x")
widgets["select_cpt"].param.watch(update_map_rtp, "value")
widgets["select_param"].param.watch(update_map_param, "value")

dashboard = pn.Row(
    pn.Column(
        widgets["select_param"],
        "Click on the map to select a GVO",
        panes["hv-map"],
        pn.Row(widgets["select_cpt"], loading_indicator),
    ),
    pn.Column(panes["hv-gvo-line"], width=400),
)

In [ ]:
dashboard.servable()

---

TODO:

- integrate with data loaded via vires instead
- add model predictions
- get ideas for advanced interactivity (e.g. fitting SH models)
- eventually serve as standalone dashboard:
    - https://panel.holoviz.org/user_guide/Deploy_and_Export.html
    - can be done for an individual user with `panel serve <notebook.ipynb>`
    - example from docker container:
      `docker run --rm -it -u jovyan -p 5006:5006 -v ~/Temp/vo-dashboard:/home/jovyan registry.gitlab.eox.at/esa/vires_vre_ops/vre-swarm-notebook:0.7.5 panel serve vo-dashboard.ipynb`